## An Introduction to the Olm python package

The python [*olm*](https://github.com/CovingtonResearchGroup/olm) package provides a variety of functions to make basic geochemical calculations for carbonate waters. 

## Solution objects

Many of the functions either create or make use of a [`solution`](https://olm.readthedocs.io/en/master/olm.general.solution.html#olm.general.solution) object, which stores the chemical concentrations and other parameters (e.g. temperature, conductance) of the solution of interest.

You can obtain the list of possible ions within a solution object using the `olm.general.getProperties()`

In [1]:
from olm.general import getProperties

In [2]:
#The dictionary returned contains the available ions, their weights, charges, and radii.
getProperties()

{'H': {'weight': 1.00794, 'charge': 1.0, 'radius': 9.0},
 'Ca': {'weight': 40.08, 'charge': 2.0, 'radius': 6.0},
 'Cl': {'weight': 35.46, 'charge': -1.0, 'radius': 3.0},
 'CO3': {'weight': 60.01, 'charge': -2.0, 'radius': 4.5},
 'HCO3': {'weight': 61.02, 'charge': -1.0, 'radius': 4.0},
 'K': {'weight': 39.1, 'charge': 1.0},
 'Mg': {'weight': 24.32, 'charge': 2.0, 'radius': 8.0},
 'Na': {'weight': 22.99, 'charge': 1.0, 'radius': 4.0},
 'NO3': {'weight': 62.01, 'charge': -1.0},
 'PO4': {'weight': 94.98, 'charge': -3.0},
 'SO4': {'weight': 96.06, 'charge': -2.0, 'radius': 4.0},
 'H2CO3': {'weight': 62.03, 'charge': 0.0},
 'OH': {'weight': 17.01, 'charge': -1.0, 'radius': 3.0},
 'CO2': {'weight': 44.01, 'charge': 0},
 'H2CO3s': {'weight': 62.03, 'charge': 0},
 'CaCO3': {'weight': 100.09, 'charge': 0}}

You can create solution objects manually, as below, though this would be a relatively uncommon workflow.

In [23]:
from olm.general import solution

#We will create a solution with a few of the species listed above
ions = ['Ca','Cl','H2CO3']
concentrations = [15,2.5,0.0006]
sol = solution(ions,concentrations, units='mg/L')


Solutions have several convenience functions attached to them.

In [29]:
#You can retrieve the concentration in mg/L, mol/L or meq using mg(), mol(), or meq()
print("Cl concentration is =", sol.mg('Cl'), 'mg/L' )
print("Cl concentration is =", sol.mol('Cl'), 'mol/L' )
print("Cl concentration is =", sol.meq('Cl'), 'meq/L' )
print(" ")

#You can retrieve the activity of a given species
print("Activity of Ca is =",sol.activity('Ca'),"mol/L")

#You can calculate the charge balance
print("The charge balance of the solution is", sol.charge_balance(), '%')
#Note that the charge doesn't balance well because we used a random set of species concentrations
# that aren't physically realistic.

Cl concentration is = 2.5 mg/L
Cl concentration is = 7.050197405527355e-05 mol/L
Cl concentration is = -0.07050197405527356 meq/L
 
Activity of Ca is = 0.0003230193806309863 mol/L
The charge balance of the solution is 82.78350515463917 %


## The olm.general module

In addition to defining the `solution` object, the [`olm.general`](https://olm.readthedocs.io/en/master/olm.html#olm-general-module) module contains functions that are useful for some common geochemical calculations. A few examples are given below.

#### Temperature conversion

In [30]:
from olm.general import CtoK, KtoC
print('25 C is equal to', CtoK(25), 'K')


25 C is equal to 298.15 K


#### Concentration conversions

In [32]:
from olm.general import molL_to_mgL, mgL_to_molL, mmolL_to_meqL, molL_to_meqL

#Any of the above functions can be used to convert concentration units for any 
#species in the getProperties() dictionary shown above using the string that defines 
#the species.

#For example
print("15 mg/L Ca is", mgL_to_molL(15, 'Ca'),'mol/L Ca' )


15 mg/L Ca is 0.00037425149700598805 mol/L Ca
